In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import pandas as pd
from PIL import Image

class GlassesDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = str(self.dataframe.iloc[idx, 0])  # Convert to string
        try:
            image = Image.open(img_name).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            return None, None  # Return None if there's an error loading the image

        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Load data
train_df = pd.read_csv('C:/Users/sanda/OneDrive - University of Central Florida/UCF/4_Spring_2024/Courses/STA6367 Statistical Methodology for Data Science II/Final Project/glasses/train.csv')
train_directory = 'C:/Users/sanda/OneDrive - University of Central Florida/UCF/4_Spring_2024/Courses/STA6367 Statistical Methodology for Data Science II/Final Project/glasses/faces-spring-2020/faces-spring-2020/face-'
train_df['path'] = train_df['id'].apply(lambda x: train_directory + str(x) + '.png').tolist()
train_df = train_df[['path', 'glasses']]

# Create datasets
train_dataset = GlassesDataset(train_df, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the feedforward neural network architecture
class FFNN(nn.Module):
    def __init__(self):
        super(FFNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3 * 64 * 64, 500)  # Adjusted input size for 64x64 images
        self.fc2 = nn.Linear(500, 1)  # Output dimension is 1 for binary classification
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x

# Initialize the model
model = FFNN()

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Training the model
train_losses = []  # to track the training loss over epochs
for epoch in range(20):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().unsqueeze(1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'Epoch [{epoch + 1}/{20}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            train_losses.append(running_loss / 100)
            running_loss = 0.0

print('Finished Training')


Epoch [1/20], Step [100/141], Loss: 36.4492
Epoch [2/20], Step [100/141], Loss: 36.8125
Epoch [3/20], Step [100/141], Loss: 36.1875
Epoch [4/20], Step [100/141], Loss: 36.9062
Epoch [5/20], Step [100/141], Loss: 36.2500
Epoch [6/20], Step [100/141], Loss: 36.4375
Epoch [7/20], Step [100/141], Loss: 36.5938
Epoch [8/20], Step [100/141], Loss: 37.2188
Epoch [9/20], Step [100/141], Loss: 36.2188
Epoch [10/20], Step [100/141], Loss: 36.5938
Epoch [11/20], Step [100/141], Loss: 37.0312
Epoch [12/20], Step [100/141], Loss: 36.2500
Epoch [13/20], Step [100/141], Loss: 36.5625
Epoch [14/20], Step [100/141], Loss: 36.5312
Epoch [15/20], Step [100/141], Loss: 36.5625
Epoch [16/20], Step [100/141], Loss: 36.6875
Epoch [17/20], Step [100/141], Loss: 36.4062
Epoch [18/20], Step [100/141], Loss: 36.7500
Epoch [19/20], Step [100/141], Loss: 37.4062
Epoch [20/20], Step [100/141], Loss: 36.6875
Finished Training


In [13]:
# Function to calculate accuracy
def calculate_accuracy(loader, model):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            outputs = model(images)
            predicted = (outputs > 0.5).float()  # Predicted as positive if output > 0.5
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    return correct / total

# Calculate training accuracy
train_accuracy = calculate_accuracy(train_loader, model)
print(f"Training Accuracy: {train_accuracy}")

Training Accuracy: 0.6346666666666667


# Test

In [14]:
class GlassesTestDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = str(self.dataframe.iloc[idx, 0])  # Convert to string
        try:
            image = Image.open(img_name).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_name}: {e}")
            return None

        if self.transform:
            image = self.transform(image)

        return image


In [15]:

test_df = pd.read_csv('C:/Users/sanda/OneDrive - University of Central Florida/UCF/4_Spring_2024/Courses/STA6367 Statistical Methodology for Data Science II/Final Project/glasses/test.csv')
test_directory = 'C:/Users/sanda/OneDrive - University of Central Florida/UCF/4_Spring_2024/Courses/STA6367 Statistical Methodology for Data Science II/Final Project/glasses/faces-spring-2020/test/face-'
test_df['path'] = test_df['id'].apply(lambda x: test_directory + str(x) + '.png').tolist()
test_df = test_df[['path']]


In [16]:
test_dataset = GlassesTestDataset(test_df, transform=transform)

# Create data loader for testing
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Make predictions on the test set
test_predictions = []

with torch.no_grad():
    for images in test_loader:
        outputs = model(images)
        predictions = torch.round(outputs)  # Round the probabilities to get binary predictions
        test_predictions.extend(predictions.numpy().flatten())

# Convert predictions to binary values
test_predictions = [int(pred) for pred in test_predictions]

# Print the predictions
print(test_predictions)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
ytest = pd.read_csv('C:/Users/sanda/OneDrive - University of Central Florida/UCF/4_Spring_2024/Courses/STA6367 Statistical Methodology for Data Science II/Final Project/glasses/ytrain.csv', header=None)

In [20]:
vector = ytest.to_numpy().flatten()

In [21]:
pred_vals = 0

# Iterate over each pair of elements in test_predictions and vector
for pred, true_val in zip(test_predictions, vector):
    if pred == true_val:
        pred_vals += 1

print(pred_vals/len(test_predictions)*100)


58.8
